In [2]:
import requests
import json
import pandas as pd
import numpy as np 
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from fuzzywuzzy import fuzz

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
vacancies = []

for page_number in tqdm(range(30)):
    try:
        response = requests.get("https://api.hh.ru/vacancies",
                                params={
                                    "text": 'NAME:Повар',
                                    "page": page_number,
                                    "area": [32, 23, 112, 52, 66],
                                    "per_page" : 100}
                               )
        json_response = response.json()

        vacancies += json_response["items"]
    except Exception as ex:
        print(ex)


  0%|          | 0/30 [00:00<?, ?it/s]

 73%|███████▎  | 22/30 [00:06<00:01,  6.48it/s]

'items'
'items'


 80%|████████  | 24/30 [00:06<00:00,  7.11it/s]

'items'
'items'


...

'items'


 97%|█████████▋| 29/30 [00:07<00:00,  6.15it/s]

'items'
'items'


100%|██████████| 30/30 [00:07<00:00,  4.07it/s]

'items'


In [4]:
vacancies_dataset = {
    "vacancy_name": [],
    "employer_name": [],
    "city": [],
    "salary": []
}

for vacancy in vacancies:
    vacancies_dataset["vacancy_name"].append(vacancy["name"])
    vacancies_dataset["employer_name"].append(vacancy["employer"]["name"])
    
    try:
        vacancies_dataset["city"].append(vacancy.get("address", {}).get("city", "no city"))
    except:
        vacancies_dataset["city"].append("no city")
        
    salary_from = None
    salary_to = None
    salary_currency = None
    
    try:
        salary_from = vacancy.get('salary', {}).get('from', None)
    except:
        pass
    
    try:
        salary_to = vacancy.get('salary', {}).get('to', None)
    except:
        pass
    
    try:
        salary_currency = vacancy.get('salary', {}).get('currency', None)
    except:
        pass

    
    if salary_currency != 'RUR' or (salary_from is None and salary_to is None):
        vacancies_dataset["salary"].append(None)
    elif salary_from is not None and salary_to is not None:
        vacancies_dataset["salary"].append((salary_from + salary_to) / 2.0)
    elif salary_from is not None:
        vacancies_dataset["salary"].append(salary_from)
    elif salary_to is not None:
        vacancies_dataset["salary"].append(salary_to)
    
vacancies_dataset = pd.DataFrame(vacancies_dataset)

print(vacancies_dataset.salary.mean())

47304.91082317073


In [5]:
vacancies_dataset

,vacancy_name,employer_name,city,salary
0,Шеф-повар,заведение в центре города,no city,87500.0
1,Повар,Про-Мис,Нижний Новгород,40000.0
2,"Шеф-повар на новое направление кейтеринга, кот...",Шатер НН,Нижний Новгород,85000.0
3,Помощник повара,Ярославский завод напитков,Ярославль,32000.0
4,Повар-универсал на теплоход,Авто-Респект НН,no city,50000.0
...,...,...,...,...
693,Повар-кассир в Ресторан KFC,Телетел,no city,30000.0
694,"Повар в ресторан ""Кофемания"" (г. Москва)",Кофемания,no city,70000.0
695,"Повар в ресторан ""Кофемания"" (г. Москва)",Кофемания,no city,70000.0
696,"Повар в ресторан ""Кофемания"" (г. Москва)",Кофемания,no city,70000.0
